In [1]:
 COLAB = True

if COLAB:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  local_path = '/content/drive/My Drive/ProyectoIA2/generative-spectral-datasets'
else:
  local_path = r'G:\Mi unidad\ProyectoIA2\generative-spectral-datasets'

import os 
os.chdir(local_path)

Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np

from models.main import make_autoencoder, make_discriminator
from models.metrics import PSNR, SSIM
from data import load_dataset

input_shape = (512,512,31)

In [3]:
BATCH_SIZE = 3

train_params = dict(
    batch_size = BATCH_SIZE,
    shuffle = True,
    cache = True
)

test_params = dict(
    batch_size = BATCH_SIZE,
    cache = True
)

train_ds , test_ds = load_dataset( train_params = train_params, test_params = test_params)

In [4]:
FACTOR = [ 1 , 1 , 1/2 , 1/2 , 1/4, 1/8]

features= 64

autoencoder = make_autoencoder(input_shape, features=features, factors=FACTOR)
mse_loss = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(1e-3)

metrics = [PSNR(), SSIM()]
autoencoder.compile( optimizer=optimizer, loss=mse_loss, metrics=metrics)

In [5]:
autoencoder.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 512, 31)]    0         
_________________________________________________________________
encoder (Encoder)            (None, 512, 512, 8)       59184     
_________________________________________________________________
generator (Generator)        (None, 512, 512, 31)      59230     
Total params: 118,414
Trainable params: 118,414
Non-trainable params: 0
_________________________________________________________________


In [6]:
!ls

autoencoder.h5	data  data.py  models  notebooks  __pycache__


In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau


WEIGHTS_PATH = "autoencoder.h5"

callbacks = [
            ModelCheckpoint(WEIGHTS_PATH, monitor='val_psnr', verbose=1,save_best_only=True, save_weights_only=True,  mode="max",),
            ReduceLROnPlateau(monitor="val_psnr", factor=0.9, patience=15, verbose=1, mode="max", cooldown=15)
]


In [8]:
!ls

autoencoder.h5	data  data.py  models  notebooks  __pycache__


In [ ]:
autoencoder.fit( train_ds , epochs=1000, validation_data=test_ds, callbacks=callbacks)

Epoch 1/1000
9/9 [==============================] - 93s 2s/step - loss: 0.0109 - psnr: 20.5862 - ssim: 0.5445 - val_loss: 0.0067 - val_psnr: 22.1069 - val_ssim: 0.6655

Epoch 00001: val_psnr improved from -inf to 22.10690, saving model to autoencoder.h5
Epoch 2/1000
9/9 [==============================] - 11s 1s/step - loss: 0.0052 - psnr: 23.6883 - ssim: 0.7032 - val_loss: 0.0039 - val_psnr: 24.4049 - val_ssim: 0.7052

Epoch 00002: val_psnr improved from 22.10690 to 24.40492, saving model to autoencoder.h5
Epoch 3/1000
9/9 [==============================] - 11s 1s/step - loss: 0.0032 - psnr: 25.7682 - ssim: 0.7615 - val_loss: 0.0023 - val_psnr: 26.7243 - val_ssim: 0.7696

Epoch 00003: val_psnr improved from 24.40492 to 26.72427, saving model to autoencoder.h5
Epoch 4/1000
9/9 [==============================] - 11s 1s/step - loss: 0.0024 - psnr: 27.0574 - ssim: 0.7929 - val_loss: 0.0016 - val_psnr: 28.2888 - val_ssim: 0.7729

Epoch 00004: val_psnr improved from 26.72427 to 28.28883, sav